In [2]:
!pip install pypdf python-dotenv

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from pypdf import PdfReader, PdfWriter

def extract_first_page(input_path, output_path):
    # Create a reader object
    reader = PdfReader(input_path)
    
    # Create a writer object
    writer = PdfWriter()
    
    # Add the first page (index 0)
    if len(reader.pages) > 0:
        writer.add_page(reader.pages[0])
    else:
        raise ValueError("The PDF file is empty.")
    
    # Write the single-page PDF to the output file
    with open(output_path, "wb") as f:
        writer.write(f)
    
    print(f"First page saved successfully as '{output_path}'")

# Example usage
extract_first_page("Worthy Payments Website (Copy File).pdf", "first_page.pdf")


First page saved successfully as 'first_page.pdf'


In [3]:
import requests
import json
import os


def get_json(figma_url, output_file, FIGMA_ACCESS_TOKEN):
    # === STEP 1: Extract File Key from URL ===
    # The file key is the part after /design/ or /file/
    import re
    match = re.search(r"/(?:file|design)/([a-zA-Z0-9]+)", figma_url)
    if not match:
        raise ValueError("Could not extract file key from the given URL.")
    file_key = match.group(1)
    
    
    def extract_node_id(figma_url: str) -> str | None:
        """
        Extracts the node-id value from a Figma URL.
    
        Example:
            url = "https://www.figma.com/design/Dm0rcuxF6AYnlaHKpwCJjR/Worthy-Payments-Website?node-id=1-1718&t=jX26OpIa5zmao1s0-4"
            result = extract_node_id(url)
            print(result)  # '1-1718'
        """
        match = re.search(r"[?&]node-id=([\w-]+)", figma_url)
        return match.group(1) if match else None
        
    node_ids = extract_node_id(figma_url)
    
    
    # === STEP 2: Request the file data from Figma API ===
    api_url = f"https://api.figma.com/v1/files/{file_key}"
    if node_ids:
        api_url = f"{api_url}/nodes?ids={node_ids}"
    
    print(f"Fetching data from {api_url}")
    headers = {
        "X-Figma-Token": FIGMA_ACCESS_TOKEN
    }
    
    response = requests.get(api_url, headers=headers)
    
    # === STEP 3: Check and Save JSON ===
    if response.status_code == 200:
        data = response.json()
        
        with open(output_file, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=2, ensure_ascii=False)
        print("✅ Figma design JSON saved as figma_design.json")
    else:
        print(f"❌ Failed to fetch data: {response.status_code}")
        print(response.text)


# Replace this with your own personal access token from https://www.figma.com/developers/api
token = os.getenv("FIGMA_ACCESS_TOKEN")
base_dir = "jsons"


# Figma file URL
figma_url = "https://www.figma.com/design/Dm0rcuxF6AYnlaHKpwCJjR/Worthy-Payments-Website--Copy-File----Partial-file-saved-11-3-2025?node-id=1-2509&t=wGaOeIEkTdaj7xb7-4"
output_file = f"{base_dir}/card_two_layout.json"

get_json(figma_url, output_file, token)

Fetching data from https://api.figma.com/v1/files/Dm0rcuxF6AYnlaHKpwCJjR/nodes?ids=1-2509
✅ Figma design JSON saved as figma_design.json
